In [1]:
from utils.dataLoader import DataLoader
from utils.modelLoader import ModelLoader
from utils.SiameseModel import SiameseModel
import tensorflow as tf
import pandas as pd

### Runs Triplet data

In [2]:
runs = {
    "Siamese_MobileNetV1_Train500_All": "metadate_all_limit500.json",
    "Siamese_MobileNetV1_Train5000_All": "metadate_all_limit5000.json"
}

In [3]:
for run in runs.keys():
    train_ds, val_ds, test_ds = DataLoader().loadDatasets("img_align_celeba", runs[run], 32)
    
    model, embedding = ModelLoader().loadMobileNetV1FaceRecognition(True)
    siamese_model = SiameseModel(siamese_network=model)
    siamese_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), weighted_metrics=[])
    
    log_dir = "../logs/fit/" + run
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    siamese_model.fit(train_ds, epochs=20, validation_data=val_ds, callbacks=[tensorboard_callback])
    
    runPath = "../models/" + run
    model.save(runPath)
    embedding.save(runPath + "_embedding")

Epoch 1/20
11/11 [==============================] - 21s 1s/step - loss: 15.1831 - val_loss: 9.7778
Epoch 2/20
11/11 [==============================] - 17s 762ms/step - loss: 7.4801 - val_loss: 11.7879
Epoch 3/20
11/11 [==============================] - 7s 651ms/step - loss: 6.2252 - val_loss: 1.2277
Epoch 4/20
11/11 [==============================] - 8s 701ms/step - loss: 1.6107 - val_loss: 0.4155
Epoch 5/20
11/11 [==============================] - 8s 704ms/step - loss: 1.3626 - val_loss: 0.0000e+00
Epoch 6/20
11/11 [==============================] - 7s 619ms/step - loss: 0.9180 - val_loss: 0.3723
Epoch 7/20
11/11 [==============================] - 7s 652ms/step - loss: 0.8589 - val_loss: 0.0000e+00
Epoch 8/20
11/11 [==============================] - 7s 628ms/step - loss: 0.8390 - val_loss: 0.0884
Epoch 9/20
11/11 [==============================] - 7s 637ms/step - loss: 0.2744 - val_loss: 0.1495
Epoch 10/20
11/11 [==============================] - 7s 655ms/step - loss: 2.9813 - val_los

INFO:tensorflow:Assets written to: ../models/Siamese_MobileNetV1_Train500_All\assets


INFO:tensorflow:Assets written to: ../models/Siamese_MobileNetV1_Train500_All\assets


INFO:tensorflow:Assets written to: ../models/Siamese_MobileNetV1_Train500_All_embedding\assets


INFO:tensorflow:Assets written to: ../models/Siamese_MobileNetV1_Train500_All_embedding\assets


Epoch 1/20
110/110 [==============================] - 294s 2s/step - loss: 9.4159 - val_loss: 3.0401
Epoch 2/20
110/110 [==============================] - 192s 2s/step - loss: 2.4327 - val_loss: 0.7893
Epoch 3/20
110/110 [==============================] - 39s 290ms/step - loss: 0.9454 - val_loss: 0.3586
Epoch 4/20
110/110 [==============================] - 40s 297ms/step - loss: 0.4501 - val_loss: 0.2368
Epoch 5/20
110/110 [==============================] - 40s 295ms/step - loss: 0.3195 - val_loss: 0.2313
Epoch 6/20
110/110 [==============================] - 41s 300ms/step - loss: 0.2635 - val_loss: 0.2507
Epoch 7/20
110/110 [==============================] - 41s 303ms/step - loss: 0.2607 - val_loss: 0.2307
Epoch 8/20
110/110 [==============================] - 43s 315ms/step - loss: 0.1826 - val_loss: 0.1565
Epoch 9/20
110/110 [==============================] - 44s 321ms/step - loss: 0.2087 - val_loss: 0.1895
Epoch 10/20
110/110 [==============================] - 47s 336ms/step - loss:

INFO:tensorflow:Assets written to: ../models/Siamese_MobileNetV1_Train5000_All\assets


INFO:tensorflow:Assets written to: ../models/Siamese_MobileNetV1_Train5000_All\assets


INFO:tensorflow:Assets written to: ../models/Siamese_MobileNetV1_Train5000_All_embedding\assets


INFO:tensorflow:Assets written to: ../models/Siamese_MobileNetV1_Train5000_All_embedding\assets


In [5]:
train_ds, val_ds, test_ds = DataLoader().loadDatasets("img_align_celeba", "metadate_identity-once_limit500.json", 32)

In [ ]:
siamese_model.save("../models/Siamese_MobileNetV1_Train500")
embedding.save("../models/Siamese_MobileNetV1_Train500_embedding")

In [7]:
embedding = tf.keras.models.load_model("../models/Siamese_MobileNetV1_Train5000_All_embedding")

In [8]:
for sample in iter(test_ds):
    anchor, positive, negative = sample

    anchor_embedding, positive_embedding, negative_embedding = (
        embedding(tf.keras.applications.mobilenet.preprocess_input(anchor)),
        embedding(tf.keras.applications.mobilenet.preprocess_input(positive)),
        embedding(tf.keras.applications.mobilenet.preprocess_input(negative)),
    )
    
    cosine_similarity = tf.keras.metrics.CosineSimilarity()

    positive_similarity = cosine_similarity(anchor_embedding, positive_embedding)
    print("Positive similarity:", positive_similarity.numpy())

    negative_similarity = cosine_similarity(anchor_embedding, negative_embedding)
    print("Negative similarity", negative_similarity.numpy())
    
    print("Difference:                        ", positive_similarity.numpy()-negative_similarity.numpy())

Positive similarity: 0.9566349
Negative similarity 0.919959
Difference:                         0.03667587


In [2]:
def contrastive_loss(y, preds, margin=1):
    y = tf.cast(y, preds.dtype)
    squaredPreds = tf.keras.backend.square(preds)
    squaredMargin = tf.keras.backend.square(tf.keras.backend.maximum(margin - preds, 0))
    loss = tf.keras.backend.mean(y * squaredPreds + (1 - y) * squaredMargin) 
    return 1-loss

### Run with Pair data cropped

In [3]:
train_ds, val_ds, test_ds = DataLoader().loadDatasetsPairs("img_align_celeba", "metadate_pairs_all_limit500.json", 32, False)

model, embedding = ModelLoader().loadMobileNetV1FaceRecognitionPair()

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=contrastive_loss, optimizer=opt, metrics=["accuracy"])

log_dir = "../logs/fit/Siamese_MobileNetV1_Pairs_Train500_All_E20"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=[tensorboard_callback])

model.save("../models/Siamese_MobileNetV1_Pairs_Train200p_All_E20")
embedding.save("../models/Siamese_MobileNetV1_Pairs_Train200_All_E20_Embedding")

Epoch 1/20
478/478 [==============================] - 1211s 2s/step - loss: 0.5058 - accuracy: 0.4977 - val_loss: 0.4999 - val_accuracy: 0.5005
Epoch 2/20
478/478 [==============================] - 1088s 2s/step - loss: 0.5004 - accuracy: 0.4996 - val_loss: 0.4852 - val_accuracy: 0.5148
Epoch 3/20
478/478 [==============================] - 1037s 2s/step - loss: 0.5033 - accuracy: 0.4968 - val_loss: 0.4954 - val_accuracy: 0.5047
Epoch 4/20
478/478 [==============================] - 908s 2s/step - loss: 0.4989 - accuracy: 0.5011 - val_loss: 0.4985 - val_accuracy: 0.5017
Epoch 5/20
478/478 [==============================] - 928s 2s/step - loss: 0.5006 - accuracy: 0.4994 - val_loss: 0.4953 - val_accuracy: 0.5047
Epoch 6/20
478/478 [==============================] - 923s 2s/step - loss: 0.4980 - accuracy: 0.5020 - val_loss: 0.4999 - val_accuracy: 0.5002
Epoch 7/20
478/478 [==============================] - 931s 2s/step - loss: 0.5019 - accuracy: 0.4981 - val_loss: 0.5023 - val_accuracy: 0.4

INFO:tensorflow:Assets written to: ../models/Siamese_MobileNetV1_Pairs_Train200p_All_E20\assets


INFO:tensorflow:Assets written to: ../models/Siamese_MobileNetV1_Pairs_Train200p_All_E20\assets


INFO:tensorflow:Assets written to: ../models/Siamese_MobileNetV1_Pairs_Train200_All_E20_Embedding\assets


INFO:tensorflow:Assets written to: ../models/Siamese_MobileNetV1_Pairs_Train200_All_E20_Embedding\assets


### Run with Pair data

In [4]:
train_ds, val_ds, test_ds = DataLoader().loadDatasetsPairs("img_align_celeba", "metadate_pairs_all_limit500.json", 32)

model, embedding = ModelLoader().loadMobileNetV1FaceRecognitionPair()

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=contrastive_loss, optimizer=opt, metrics=["accuracy"])

log_dir = "../logs/fit/Siamese_MobileNetV1_Pairs_Train500_Crop_All_E20"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=[tensorboard_callback])

model.save("../models/Siamese_MobileNetV1_Pairs_Train200_Crop_All_E20")
embedding.save("../models/Siamese_MobileNetV1_Pairs_Train200_Crop_All_E20_Embedding")

Epoch 1/20
478/478 [==============================] - 1266s 2s/step - loss: 0.4988 - accuracy: 0.5023 - val_loss: 0.5005 - val_accuracy: 0.4995
Epoch 2/20
478/478 [==============================] - 1251s 2s/step - loss: 0.4997 - accuracy: 0.5004 - val_loss: 0.5148 - val_accuracy: 0.4852
Epoch 3/20
478/478 [==============================] - 1289s 2s/step - loss: 0.4968 - accuracy: 0.5032 - val_loss: 0.5047 - val_accuracy: 0.4953
Epoch 4/20
478/478 [==============================] - 1325s 2s/step - loss: 0.5011 - accuracy: 0.4989 - val_loss: 0.5017 - val_accuracy: 0.4983
Epoch 5/20
478/478 [==============================] - 1328s 2s/step - loss: 0.4994 - accuracy: 0.5006 - val_loss: 0.5047 - val_accuracy: 0.4953
Epoch 6/20
478/478 [==============================] - 1341s 2s/step - loss: 0.5020 - accuracy: 0.4980 - val_loss: 0.5002 - val_accuracy: 0.4998
Epoch 7/20
478/478 [==============================] - 1386s 2s/step - loss: 0.4981 - accuracy: 0.5019 - val_loss: 0.4977 - val_accuracy: